# Rover.com Analytics Assessment
**Andrew Nicholls** | Email: andrew.s.nicholls@gmail.com | [Github](https://github.com/Booleans)

If you are viewing this notebook in Github you may want to switch to the nbviewer page to ensure interactive charts are functioning properly: https://nbviewer.jupyter.org/github/Booleans/rover-analytics-assessment/blob/master/analysis.ipynb

### Exercise Problems
Click the hyperlink to skip to the desired assessment section.

I. [Exploring the Database](#1)  
II. [Conversations and Bookings](#2)  
III. [Recent Daily Booking Rate](#3)   
IV. [Analyzing Take Rate](#4)   
V. [New Conversation Flow](#5)